In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## **1. Скачайте набор данных Covid 19 Indian Sentiments on covid19 and lockdown по ссылке**

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"skylarpro","key":"61c6a5a697295af30a778f1880685730"}'}

In [3]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

rm: cannot remove '/root/.kaggle': No such file or directory
ref                                                                   title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                              Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23           5747        180  1.0              
surajjha101/stores-area-and-sales-data                                Supermarket store branches sales analysis            10KB  2022-04-29 11:10:16           5189        154  1.0              
devansodariya/student-performance-data                                Student Performance Dataset                           7KB  2022-05-26 13:55:09           2147

In [4]:
!kaggle datasets download "surajkum1198/twitterdata"
!unzip /content/twitterdata.zip

  0% 0.00/190k [00:00<?, ?B/s]
100% 190k/190k [00:00<00:00, 35.6MB/s]
Archive:  /content/twitterdata.zip
  inflating: finalSentimentdata2.csv  


##В датафрейме оставьте только те строки, у которых в столбце sentiment написано sad или joy

In [5]:
df = pd.read_csv("/content/finalSentimentdata2.csv")
new_df = df[(df["sentiment"] == "joy") | (df["sentiment"] == "sad")]
new_df.drop(["Unnamed: 0"], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
new_df.sentiment.unique()

array(['sad', 'joy'], dtype=object)

##Примените на этом наборе данных шаги предварительной обработки 

In [7]:
!pip install pymorphy2
!pip install -U pymorphy2-dicts-uk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 13.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.0 MB 9.2 MB/s 


In [8]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import pymorphy2


tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True,
                                                                    reduce_len=True,                                    
                                                                   )
_morph = pymorphy2.MorphAnalyzer(lang='uk')
nltk.download('stopwords')

stpword = stopwords.words('english')
def _processing_text(proc_text):
        proc_text = proc_text.lower()
        proc_text = re.sub('[^a-z]', " ", proc_text)
        proc_text = proc_text.replace("  ", " ")
        
        proc_text = re.sub(r"http\S+", "",proc_text)
        proc_text = re.sub(r'#','',proc_text)
    
        proc_text = [char for char in proc_text if char not in string.punctuation]
        proc_text = ''.join(proc_text)
        
        
        proc_text =  ' '.join([word for word in proc_text.split() if word.lower() not in stpword])
        proc_text = tokenizer.tokenize(proc_text)
        sent_lemm = []
        
        for word in proc_text:
            word_normal = _morph.parse(word)[0]
            #confidenc model to cast form word
            if word_normal.score > 0.70:
                sent_lemm.append(word_normal.normal_form)
            else:
                sent_lemm.append(word)
        proc_text = ' '.join(sent_lemm)
                
        return proc_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
new_df["text_proc"] = new_df["text"].apply(_processing_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


##Обучите классификатор

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
train, test = train_test_split(new_df,shuffle = True)

In [12]:
test["sentiment"].unique()

array(['joy', 'sad'], dtype=object)

In [13]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train["text_proc"])
X_train = vectorizer.transform(train["text_proc"])
y_train = np.array(train["sentiment"] == "joy").astype(np.uint8)

In [14]:
X_test = vectorizer.transform(test["text_proc"])
y_test = np.array(test["sentiment"] == "joy").astype(np.uint8)

In [15]:
assert X_test.shape[-1] == X_train.shape[-1]

In [16]:
clf = MultinomialNB()

clf.fit(X_train, y_train)

pred = clf.predict(X_test)

print(accuracy_score(y_test, pred))

0.8372703412073491


##Примените шаги предобработки на наборе данных COVID19 Tweets

In [17]:
!kaggle datasets download "gpreda/covid19-tweets"
!unzip /content/covid19-tweets.zip

 61% 17.0M/27.9M [00:00<00:00, 17.9MB/s]
100% 27.9M/27.9M [00:01<00:00, 29.1MB/s]
Archive:  /content/covid19-tweets.zip
  inflating: covid19_tweets.csv      


In [18]:
df_2 = pd.read_csv("/content/covid19_tweets.csv")
df_2

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,AJIMATI AbdulRahman O.,"Ilorin, Nigeria",Animal Scientist|| Muslim|| Real Madrid/Chelsea,2013-12-30 18:59:19,412,1609,1062,False,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,['WearAMask'],Twitter for Android,False
179104,Jason,Ontario,When your cat has more baking soda than Ninja ...,2011-12-21 04:41:30,150,182,7295,False,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,['COVID19'],Twitter for Android,False
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,NaN,Twitter Web App,False
179106,Gary DelPonte,New York City,"Global UX UI Visual Designer. StoryTeller, Mus...",2009-10-27 17:43:13,1338,1111,0,False,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...",['COVID19'],Twitter for iPhone,False


In [19]:
df_2["text_proc"] = df_2["text"].apply(_processing_text)
df_2["text_proc"]

0         smelled scent hand sanitizers today someone pa...
1         hey yankees yankeespr mlb made sense players p...
2         diane wdunlap realdonaldtrump trump never clai...
3         brookbanktv one gift covid give appreciation s...
4         july media bulletin novel coronavirusupdates c...
                                ...                        
179103    thanks iamohmai nominating wearamask challenge...
179104                   year insanity lol covid co np yzgn
179105    ctvnews powerful painting juan lucena tribute ...
179106    students test positive covid major university ...
179107    stop see stop sabcnews izindabanews dailysunsa...
Name: text_proc, Length: 179108, dtype: object

##С помощью обученного классификатора найдите посты из набора данных COVID19 Tweets, которые можно отнести к классу sad. Какой процент от общего количества постов они составляют

In [20]:
X_control = vectorizer.transform(df_2["text_proc"])

In [21]:
pred = clf.predict(X_control)
print("Процент грустных предложений:", (pred.shape[0] - sum(pred))/pred.shape[0])

Процент грустных предложений: 0.6541025526497979


##Проанализируйте тональность постов из набора данных COVID19 Tweets с помощью библиотеки Dostoevsky или TextBlob. Какое процентное соотношение постов разных настроений наблюдается?

In [22]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 57.3 MB/s 
     |████████████████████████████████| 6.6 MB 59.1 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [24]:
from transformers import BertTokenizer
BERT_MODEL = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = 2)
model.cuda()

In [33]:
eval_texts = df_2["text_proc"].to_numpy()

In [50]:
from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler
import torch

class TweetDataset(Dataset):

  def __init__(self,data, tokenizer,max_seq_length):
    self.text_data = data
    self.tokenizer = tokenizer
    self.max_seq_length = max_seq_length

  def __getitem__(self,idx):
        text = self.text_data[idx]
        input_ids = self.tokenizer.encode(f"[CLS] {text} [SEP]")

        if len(input_ids) > self.max_seq_length:
            input_ids = input_ids[:self.max_seq_length]


        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        attention_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (self.max_seq_length - len(input_ids))
        input_ids += padding
        attention_mask += padding
                
        return torch.tensor(input_ids), torch.tensor(attention_mask)

  def __len__(self,):
    return self.text_data.shape[0]

eval_dataset = TweetDataset(eval_texts,tokenizer,100)

In [51]:
def sempler_eval(data_train, batch_size = 16,):
    
    data_size = len(data_train)

    eval_indices = list(range(data_size))

    eval_sampler = SubsetRandomSampler(eval_indices)

    eval_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                              sampler=eval_sampler,)

    return eval_loader

eval_loader = sempler_eval(eval_dataset)

In [89]:
cls = model.classifier
cls_softmax = nn.Sequential(cls,nn.Softmax())
model.classifier = cls_softmax

In [115]:
from tqdm import tqdm
import torch.nn as nn
def evaluate(model, dataloader):
  with torch.no_grad():
    model.eval()
    
    predicted_labels = []

    for step, batch in enumerate(tqdm(dataloader, desc="Evaluation iteration")):
        batch = tuple(t.cuda() for t in batch)
        input_ids, input_mask = batch

        logits = model(input_ids, attention_mask=input_mask)
        outputs = np.argmax(logits.logits.to('cpu'), axis=1)
        predicted_labels += list(outputs)

    predicted_labels = np.array(predicted_labels)
        
    return predicted_labels

In [116]:
prediction = evaluate(model, eval_loader)

Evaluation iteration:   0%|          | 0/11195 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Evaluation iteration: 100%|██████████| 11195/11195 [18:30<00:00, 10.08it/s]


In [119]:
cls, distr_predict = np.unique(prediction, return_counts = True)

In [126]:
print(f"Положительного класса {100*(distr_predict[0]/sum(distr_predict))}%" )
print(f"Отрицательного класса {100*(distr_predict[1]/sum(distr_predict))}%")

Положительного класса 4.4163298121803605%
Отрицательного класса 95.58367018781964%
